In [2]:
from itertools import product

In [3]:
def add_button_config(config, line, button):
   entry = (line[line.index("+") + 1 : line.index(",")], line[line.index("Y+") + 2 :])
   config[button] = int(entry[0]), int(entry[1])

In [ ]:
with open("Data/Day_13_content.txt") as file:
    lines = [line.split("\n")[0] for line in file.readlines()]
    machines = []
    config = {}
    for line in lines:
        
        if line.startswith("Button A"):
            add_button_config(config, line, "A")
        elif line.startswith("Button B"):
            add_button_config(config, line, "B")
        elif line.startswith("Prize"):
            config["Prize"] = int(line[line.index("X=") + 2 : line.index(",")]), int(line[line.index("Y=") + 2 :])
            
        if "A" in config and "B" in config and "Prize" in config:
            machines.append(config.copy())
            config.clear()

In [ ]:
machines[:5]

## Part 1

In [15]:
results = []  # To store results for each machine
total_tokens = 0
total_prizes = 0

In [ ]:
for i, config in enumerate(machines):
    print(config["Prize"])

In [17]:
for i, config in enumerate(machines):
    # Extract machine config
    ax, ay = config["A"]
    bx, by = config["B"]
    px, py = config["Prize"]
    
    min_cost = float("inf")  # Initialize with arbitarily high min_cost
    prize_won = False
    
    # Brute force over possible button presses
    for a, b in product(range(101), repeat = 2):
        
        # Check if combination aligns claw with prize
        if ax * a + bx * b == px and ay * a + by * b == py:
            # Calculate cost
            cost = 3 * a + b
                
            if cost < min_cost:
                min_cost = cost
                prize_won = True
                
    if prize_won:
        results.append((i + 1, min_cost))
        total_tokens += min_cost
        total_prizes += 1
    else:
        results.append((i + 1, None))  # No solution for this machine

In [18]:
# Display results
print(f"Total prizes won: {total_prizes}")
print(f"Total tokens spent: {total_tokens}")

Total prizes won: 169
Total tokens spent: 37680


## Part 2 (Incomplete)

In [3]:
import math

In [9]:
def extended_gcd(a, b):
        """Extended Euclidean algorithm to find x, y such that ax + by = gcd(a, b)."""
        if b == 0: return a, 1, 0
        g, x1, y1 = extended_gcd(b, a % b)
        x = y1
        y = x1 - (a // b) * y1
        return g, x, y

In [10]:
def find_solution(ax, bx, px, ay, by, py):
    """Find non-negative integers a, b satisfying the equations for extended px and py."""
    
    print(f"  Axes Coefficients: ax={ax}, bx={bx}, px={px}; ay={ay}, by={by}, py={py}")
    
    g_x, x1, y1 = extended_gcd(ax, bx)
    g_y, x2, y2 = extended_gcd(ay, by)
    
    print(f"  Extended GCD Results: g_x={g_x}, x1={x1}, y1={y1}; g_y={g_y}, x2={x2}, y2={y2}")
    
    # Check if solutions exist and coefficients match
    if px % g_x != 0 or py % g_y != 0:
        print("  No solution due to non-matching gcd")
        return None
        
    # Ensure gcd values match before proceeding
    elif g_x != g_y:
        
        print("  Scaled coefficients due to inequality")
        lcm_g = (g_x * g_y) // math.gcd(g_x, g_y)
        scale_g_x = lcm_g // g_x
        scale_g_y = lcm_g // g_y
        
        g_x, x1, y1 = extended_gcd(ax * scale_g_x, bx * scale_g_x)
        g_y, x2, y2 = extended_gcd(ay * scale_g_y, by * scale_g_y)

        if scale_g_x != scale_g_y:
            print("  No solution due to non-matching coefficients")
            return None

    # Scale solutions for the x-axis equation
    scale_x = px // g_x
    x1 *= scale_x
    y1 *= scale_x

    # Scale solutions for the y-axis equation
    scale_y = py // g_y
    x2 *= scale_y
    y2 *= scale_y
    print(f"  Scaled Solutions: x1={x1}, y1={y1}; x2={x2}, y2={y2}")

    # Find a valid (a, b) pair from combined solutions
    k_x = bx // g_x
    k_y = by // g_y

    # Align x1, y1 to non-negative values using the linear congruence method
    t_x = (k_x - x1 % k_x) % k_x
    t_y = (k_y - x2 % k_y) % k_y
    print(f"  Modulo Adjustments: t_x={t_x}, t_y={t_y}")
    
    if t_x != t_y:
        print("  No solution due to non-matching modulo adjustments")
        return None
    
    t = max(t_x, t_y)
    a = x1 + t * k_x
    b = y1 + t * k_y
    print(f"  Final Values: a={a}, b={b}")
    
    if a < 0 or b < 0:
        print("  No solution due to negative a, b values")
        return None

    if ax * a + bx * b != px or ay * a + by * b != py:
        print("  No solution due to final validation")
        return None

    return a, b


In [11]:
machines_test = [
    {"A": (94, 34), "B": (22, 67), "Prize": (8400, 5400)},
    {"A": (26, 66), "B": (67, 21), "Prize": (12748, 12176)},
    {"A": (17, 86), "B": (84, 37), "Prize": (7870, 6450)},
    {"A": (69, 23), "B": (27, 71), "Prize": (18641, 10279)}
    ]

In [12]:
results = []  # To store results for each machine
total_tokens = 0
total_prizes = 0

In [13]:
for i, config in enumerate(machines_test):
    machine_num = i + 1
    solution_found = False
    print(f"Machine {i+1}:")
    
    # Extract machine config
    ax, ay = config["A"]
    bx, by = config["B"]
    px, py = config["Prize"]
    
    px += 10 ** 13
    py += 10 ** 13
    
    solution = find_solution(ax, bx, px, ay, by, py)
    if solution is not None:
        solution_found = True
    
    g_x, x1, y1 = extended_gcd(ax, bx)
    g_y, x2, y2 = extended_gcd(ay, by)

    if solution_found:
        a, b = solution
        cost = 3 * a + b
        results.append((machine_num, cost))
        total_tokens += cost
        total_prizes += 1
    else:
        results.append((machine_num, None))
        
    print(f"  Solution Exists: {'Yes' if solution_found else 'No'}")

Machine 1:
  Axes Coefficients: ax=94, bx=22, px=10000000008400; ay=34, by=67, py=10000000005400
  Extended GCD Results: g_x=2, x1=4, y1=-17; g_y=1, x2=2, y2=-1
  Scaled coefficients due to inequality
  No solution due to non-matching coefficients
  Solution Exists: No
Machine 2:
  Axes Coefficients: ax=26, bx=67, px=10000000012748; ay=66, by=21, py=10000000012176
  Extended GCD Results: g_x=1, x1=-18, y1=7; g_y=3, x2=1, y2=-3
  Scaled coefficients due to inequality
  No solution due to non-matching coefficients
  Solution Exists: No
Machine 3:
  Axes Coefficients: ax=17, bx=84, px=10000000007870; ay=86, by=37, py=10000000006450
  Extended GCD Results: g_x=1, x1=5, y1=-1; g_y=1, x2=-3, y2=7
  Scaled Solutions: x1=50000000039350, y1=-10000000007870; x2=-30000000019350, y2=70000000045150
  Modulo Adjustments: t_x=38, t_y=29
  No solution due to non-matching modulo adjustments
  Solution Exists: No
Machine 4:
  Axes Coefficients: ax=69, bx=27, px=10000000018641; ay=23, by=71, py=100000000

In [14]:
# Display results
print(f"Total prizes won: {total_prizes}")
print(f"Total tokens spent: {total_tokens}")

Total prizes won: 0
Total tokens spent: 0


#### Test Output (Incorrect)

In [ ]:
from math import gcd

def solve_claw_machine(machine_configs):
    def extended_gcd(a, b):
        """Extended Euclidean algorithm to find x, y such that ax + by = gcd(a, b)."""
        if b == 0:
            return a, 1, 0
        g, x1, y1 = extended_gcd(b, a % b)
        x = y1
        y = x1 - (a // b) * y1
        return g, x, y

    def find_solution(ax, bx, px, ay, by, py):
        """Find non-negative integers a, b satisfying the equations."""
        g_x, x1, y1 = extended_gcd(ax, bx)
        g_y, x2, y2 = extended_gcd(ay, by)

        # Check if solutions exist
        if px % g_x != 0 or py % g_y != 0:
            return None

        # Scale solutions for the x-axis equation
        scale_x = px // g_x
        x1 *= scale_x
        y1 *= scale_x

        # Scale solutions for the y-axis equation
        scale_y = py // g_y
        x2 *= scale_y
        y2 *= scale_y

        # Check coefficient alignment
        if g_x != g_y:
            return None

        # Align x1 and x2 using linear congruence
        k_x = bx // g_x
        k_y = by // g_y

        # Adjust x1 and x2 to find non-negative solutions
        t_x = (k_x - x1 % k_x) % k_x
        t_y = (k_y - x2 % k_y) % k_y

        if t_x != t_y:
            return None

        # Adjust using the maximum of the offsets
        t = max(t_x, t_y)
        a = x1 + t * k_x
        b = y1 + t * k_y

        # Verify final solution
        if a < 0 or b < 0 or ax * a + bx * b != px or ay * a + by * b != py:
            return None

        return a, b

# Input: List of machine configurations
machines_test = [
    {"A": (94, 34), "B": (22, 67), "Prize": (10000000008400, 10000000005400)},
    {"A": (26, 66), "B": (67, 21), "Prize": (10000000012748, 10000000012176)},
    {"A": (17, 86), "B": (84, 37), "Prize": (10000000007870, 10000000006450)},
    {"A": (69, 23), "B": (27, 71), "Prize": (10000000018641, 10000000010279)}
]

# Solve the problem
results, total_prizes, total_tokens = solve_claw_machine(machines_test)

TypeError: cannot unpack non-iterable NoneType object

In [ ]:

# Display results
print("Results for each machine:")
for machine_id, cost in results:
    if cost is not None:
        print(f"Machine {machine_id}: Prize won with {cost} tokens")
    else:
        print(f"Machine {machine_id}: No solution")

print(f"\nTotal prizes won: {total_prizes}")
print(f"Total tokens spent: {total_tokens}")
